In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from mordred import Calculator, descriptors

**Чтение данных**

In [10]:
def chunk_read_csv(file_path : str, chunk_size: int):
    df = pd.DataFrame()
    with pd.read_csv(file_path, chunksize=chunk_size) as reader:
        df = pd.concat(reader)
    return df

In [31]:
# Загрузка таблицы
file_path = '../data/SMILES_Big_Data_Set.csv'
df = chunk_read_csv(file_path, chunk_size=200)
df

,SMILES,pIC50,mol,num_atoms,logP
0,O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1,4.26,<rdkit.Chem.rdchem.Mol object at 0x7f59df45bc30>,25,4.15910
1,O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...,4.34,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c9e0>,36,3.67430
2,NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO,4.53,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cac0>,23,1.53610
3,NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21,4.56,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cba0>,22,0.95100
4,CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1,4.59,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c7b0>,21,3.21300
...,...,...,...,...,...
16082,S=C(NN=C(c1ccccn1)c1ccccn1)Nc1ccccc1,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f59a314ed50>,24,3.21560
16083,S=C=NCCCCCCCCCCc1ccccc1,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f59a314edc0>,19,5.45270
16084,S=C=NCCCCCCCCc1ccccc1,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f59a314ee30>,17,4.67250
16085,S=c1[nH]nc(Cn2ccc3ccccc32)n1-c1ccccc1,0.00,<rdkit.Chem.rdchem.Mol object at 0x7f59a314eea0>,22,3.93289


**Информация о данных**

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16087 entries, 0 to 16086
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SMILES     16087 non-null  object 
 1   pIC50      15037 non-null  float64
 2   mol        16087 non-null  object 
 3   num_atoms  16087 non-null  int64  
 4   logP       16087 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 628.5+ KB


In [33]:
df.describe()

,pIC50,num_atoms,logP
count,15037.000000,16087.000000,16087.000000
mean,0.998739,18.749984,2.256600
std,2.479588,8.428888,1.609861
min,0.000000,3.000000,-5.395600
25%,0.000000,13.000000,1.214900
50%,0.010000,17.000000,2.163200
75%,0.130000,23.000000,3.232440
max,10.970000,85.000000,15.879200


**Поиск пропущенных значений**

In [34]:
na_values = df.isna().sum()
na_values[na_values > 0]

pIC50    1050
dtype: int64

Пропущенно 1050 значений в столбце 'plC50' (т.е. 1050 значений в столбце 'pIC50' == NaN)

Удалим эти вещества, содержащие пустые значения. В дальнейшем они могут помешать кластеризации

In [35]:
df = df.dropna()

Или же можно добавить всем им флаг, что это вещество содержит неизвестное значение PIC50

In [15]:
# Создаем бинарный флаг для пропущенных значений (0 - не пропущен, 1 - пропущен)
df.insert(2, 'pIC50_missing', df['pIC50'].isna().astype(int), True)
# Заменяем пропуски специальным значением (вне нормального диапазона)
df['pIC50'] = df['pIC50'].fillna(-999)

In [16]:
df.head()

,SMILES,pIC50,pIC50_missing,mol,num_atoms,logP
0,O=S(=O)(Nc1cccc(-c2cnc3ccccc3n2)c1)c1cccs1,4.26,0,<rdkit.Chem.rdchem.Mol object at 0x7f59df45bc30>,25,4.1591
1,O=c1cc(-c2nc(-c3ccc(-c4cn(CCP(=O)(O)O)nn4)cc3)...,4.34,0,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c9e0>,36,3.6743
2,NC(=O)c1ccc2c(c1)nc(C1CCC(O)CC1)n2CCCO,4.53,0,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cac0>,23,1.5361
3,NCCCn1c(C2CCNCC2)nc2cc(C(N)=O)ccc21,4.56,0,<rdkit.Chem.rdchem.Mol object at 0x7f59a320cba0>,22,0.9510
4,CNC(=S)Nc1cccc(-c2cnc3ccccc3n2)c1,4.59,0,<rdkit.Chem.rdchem.Mol object at 0x7f59a320c7b0>,21,3.2130


**Проверка на уникальность**

In [43]:
# duplicates = df[df.duplicated(subset=['SMILES'], keep=False)].drop('mol', axis=1)
duplicates = df[df.duplicated(subset=['SMILES'], keep=False)]
duplicates_sorted = duplicates.sort_values(by='SMILES')
print(f"Кол-во неуникальных SMILES: {duplicates_sorted['SMILES'].unique().shape[0]}")
duplicates_sorted

Кол-во неуникальных SMILES: 153


,SMILES,pIC50,mol,num_atoms,logP
1517,C=CC1(O)CCC(n2cc(C(N)=O)c(Nc3ccc(Cl)cc3)n2)C(C...,8.22,<rdkit.Chem.rdchem.Mol object at 0x7f59a295d0e0>,27,3.16088
1128,C=CC1(O)CCC(n2cc(C(N)=O)c(Nc3ccc(Cl)cc3)n2)C(C...,7.60,<rdkit.Chem.rdchem.Mol object at 0x7f59a2952500>,27,3.16088
1889,CC(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1,10.40,<rdkit.Chem.rdchem.Mol object at 0x7f59a29674c0>,24,2.05260
1865,CC(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1,10.03,<rdkit.Chem.rdchem.Mol object at 0x7f59a2966a40>,24,2.05260
162,CC(=O)NC1CCc2ccc(Oc3cnc4[nH]cc(C(=O)NC(C)C)c4n...,5.77,<rdkit.Chem.rdchem.Mol object at 0x7f59a31dfa70>,29,3.01180
...,...,...,...,...,...
757,[C-]#[N+]C1CC(OCC2CC2)CCC1n1cc(C(N)=O)c(Nc2ccn...,7.12,<rdkit.Chem.rdchem.Mol object at 0x7f59a2948190>,29,3.06779
1419,[C-]#[N+]C1CC(OCC2CC2)CCC1n1cc(C(N)=O)c(Nc2ccn...,8.05,<rdkit.Chem.rdchem.Mol object at 0x7f59a295a570>,29,3.06779
569,[C-]#[N+]C1CC(OCC2CC2)CCC1n1cc(C(N)=O)c(Nc2ccn...,6.82,<rdkit.Chem.rdchem.Mol object at 0x7f59a2942e30>,29,3.06779
643,[C-]#[N+]C1CCCCC1n1cc(C(N)=O)c(Nc2ccc(CC(=O)O)...,6.96,<rdkit.Chem.rdchem.Mol object at 0x7f59a2944f20>,27,2.75569


Всего в этой таблице 395 вещств, имеющих дубликаты SMILES с другими значениями в остальных столбцах

Давайте посмотрим, какие значения различаются в этих столбцах

In [44]:
# Группировка и подсчет кол-ва различий в каждом столбце для каждого SMILES
grouped = duplicates.groupby('SMILES').agg({
    'pIC50': 'nunique',
    'num_atoms': 'nunique',
    'logP': 'nunique',
    'mol' : 'nunique'
}).reset_index()

conflicts = {
    'pIC50': grouped[grouped['pIC50'] > 1],
    'num_atoms': grouped[grouped['num_atoms'] > 1],
    'logP': grouped[grouped['logP'] > 1],
    'mol': grouped[grouped['mol'] > 1],
    # 'other': grouped[grouped['pIC50'] == 1 & grouped['num_atoms'] == 1 & grouped['logP'] == 1 & grouped['mol'] == 1],
}

print(f"SMILES с противоречиями в pIC50: {conflicts['pIC50'].shape[0]}")
print(f"SMILES с противоречиями в num_atoms: {conflicts['num_atoms'].shape[0]}")
print(f"SMILES с противоречиями в logP: {conflicts['logP'].shape[0]}")
print(f"SMILES с противоречиями в mol: {conflicts['mol'].shape[0]}")
grouped

SMILES с противоречиями в pIC50: 153
SMILES с противоречиями в num_atoms: 0
SMILES с противоречиями в logP: 0
SMILES с противоречиями в mol: 153


,SMILES,pIC50,num_atoms,logP,mol
0,C=CC1(O)CCC(n2cc(C(N)=O)c(Nc3ccc(Cl)cc3)n2)C(C...,2,1,1,2
1,CC(=O)N1CCC(Nc2ncccc2-c2cnc3[nH]ccc3n2)C1,2,1,1,2
2,CC(=O)NC1CCc2ccc(Oc3cnc4[nH]cc(C(=O)NC(C)C)c4n...,2,1,1,2
3,CC(=O)c1ccc(Nc2nn(C3CCC(N4CCC4)CC3C#N)cc2C(N)=...,2,1,1,2
4,CC(C)(C(=O)O)c1ccc(Nc2nn(C3CCC(N4CCC4)CC3C#N)c...,2,1,1,2
...,...,...,...,...,...
148,O=C1NC(=O)C(c2c[nH]c3ccccc23)=C1Nc1cccc(OC2CCN...,2,1,1,2
149,O=c1[nH]c2cnc(-n3cnc4ccccc43)nc2n1C1CCOc2c(F)c...,3,1,1,3
150,[C-]#[N+]C1CC(N2CCC2)CCC1n1cc(Nc2ccc(OC(F)F)nc...,2,1,1,2
151,[C-]#[N+]C1CC(OCC2CC2)CCC1n1cc(C(N)=O)c(Nc2ccn...,3,1,1,3


Из последнего следует, что есть такие молекулы, у которых все свойства совпадают, кроме адреса в библиотеке RdKit и значения pIC50 (т.е. в данных нет таких дубликатов веществ, где было бы отличие в num_atoms и logP )

In [45]:
grouped[grouped['mol'] > grouped['pIC50']].shape

(0, 5)

In [46]:
df[df['SMILES'] == 'O=C(O)CNCP(=O)(O)O']

,SMILES,pIC50,mol,num_atoms,logP


29 SMILES, где отличается только столбец mol

In [47]:
# Тут совпадает все, кроме значений в столбце 'mol'
duplicates_sorted[duplicates_sorted['SMILES'] == 'CCCC(=O)O']

,SMILES,pIC50,mol,num_atoms,logP


Удалим дубликаты, где отличается только столбец mol

In [41]:
drop_df = df.drop_duplicates(subset=['SMILES', 'pIC50'], keep='first')

In [42]:
undrop_df = df
df = drop_df

In [29]:
df.to_csv('../data/data_withMissFlag_fill-999_and_duplicates.csv', index=False)

In [48]:
df.to_csv('../data/data_withDropNA_and_duplicates.csv', index=False)